In [54]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [55]:
## Read csv file
df = pd.read_csv('train_all_tasks.csv')

In [56]:
## Data pre-processing

In [73]:
# text and labels for task A
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(df['text'], df['label_sexist'], test_size=0.3, random_state=0)
X_test_a, X_dev_a, y_test_a, y_dev_a = train_test_split(X_test_a, y_test_a, test_size=0.33)

# text and labels for task B
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(df['text'], df['label_category'], test_size=0.3, random_state=0)
X_test_b, X_dev_b, y_test_b, y_dev_b = train_test_split(X_test_b, y_test_b, test_size=0.33)

In [78]:
print(Counter(y_train_a))
print(Counter(y_dev_a))
print(Counter(y_test_a))

print(Counter(y_train_b))
print(Counter(y_dev_b))
print(Counter(y_test_b))

Counter({'not sexist': 7413, 'sexist': 2387})
Counter({'not sexist': 1051, 'sexist': 335})
Counter({'not sexist': 2138, 'sexist': 676})
Counter({4: 7413, 1: 1119, 2: 808, 3: 236, 0: 224})
Counter({'none': 1039, '2. derogation': 165, '3. animosity': 126, '4. prejudiced discussions': 34, '1. threats, plans to harm and incitement': 22})
Counter({'none': 2150, '2. derogation': 306, '3. animosity': 231, '1. threats, plans to harm and incitement': 64, '4. prejudiced discussions': 63})


In [59]:
def report(model_name, ydev, pred, zero_division=1):
    print("\n=== {0} ===\n\nClassification Report:\n{1}"
          "\nConfusion Matrix:\n{2}".format(model_name,
                                            classification_report(ydev, pred, 
                                                                  zero_division=zero_division),
                                            confusion_matrix(ydev, pred)))

In [60]:
def svm_train(xtrain, ytrain, xdev, ydev, le):
  model_name = "SVM"
  model = SVC(kernel='linear')
  vec = CountVectorizer()

  vec.fit(xtrain, xdev)
  xtrain_enc = vec.transform(xtrain)
  xdev_enc = vec.transform(xdev)

  model.fit(xtrain_enc, ytrain)
  pred = model.predict(xdev_enc)
  pred = le.inverse_transform(pred)

  report(model_name, ydev, pred)

In [61]:
def majority_class(xtrain, ytrain, xdev, ydev, le):
  model_name = "majority class"
  model = DummyClassifier()
  vec = CountVectorizer()

  vec.fit(xtrain, xdev)
  xtrain_enc = vec.transform(xtrain)
  xdev_enc = vec.transform(xdev)

  model.fit(xtrain_enc, ytrain)
  pred = model.predict(xdev_enc)
  pred = le.inverse_transform(pred)

  report(model_name, ydev, pred, zero_division=0)

In [62]:
def decision_tree_train(xtrain, ytrain, xdev, ydev, le):
  model_name = "decision tree"
  model = DecisionTreeClassifier()
  vec = CountVectorizer()

  vec.fit(xtrain, xdev)
  xtrain_enc = vec.transform(xtrain)
  xdev_enc = vec.transform(xdev)

  model.fit(xtrain_enc, ytrain)
  pred = model.predict(xdev_enc)
  pred = le.inverse_transform(pred)

  report(model_name, ydev, pred)

In [63]:
## Running Task A (Binary) 

In [64]:
le = LabelEncoder()

le.fit(y_train_a)
y_train_a = le.transform(y_train_a)

In [65]:
majority_class(X_train_a, y_train_a, X_test_a, y_test_a, le)


=== majority class ===

Classification Report:
              precision    recall  f1-score   support

  not sexist       0.76      1.00      0.86      2131
      sexist       0.00      0.00      0.00       683

    accuracy                           0.76      2814
   macro avg       0.38      0.50      0.43      2814
weighted avg       0.57      0.76      0.65      2814

Confusion Matrix:
[[2131    0]
 [ 683    0]]


In [66]:
svm_train(X_train_a, y_train_a, X_test_a, y_test_a, le)


=== SVM ===

Classification Report:
              precision    recall  f1-score   support

  not sexist       0.86      0.89      0.88      2131
      sexist       0.63      0.56      0.59       683

    accuracy                           0.81      2814
   macro avg       0.75      0.73      0.74      2814
weighted avg       0.81      0.81      0.81      2814

Confusion Matrix:
[[1903  228]
 [ 300  383]]


In [67]:
decision_tree_train(X_train_a, y_train_a, X_test_a, y_test_a, le)


=== decision tree ===

Classification Report:
              precision    recall  f1-score   support

  not sexist       0.85      0.88      0.87      2131
      sexist       0.58      0.53      0.56       683

    accuracy                           0.79      2814
   macro avg       0.72      0.71      0.71      2814
weighted avg       0.79      0.79      0.79      2814

Confusion Matrix:
[[1867  264]
 [ 318  365]]


In [68]:
## Running Task B (Multi-class) 

In [74]:
le = LabelEncoder()

le.fit(y_train_b)
y_train_b = le.transform(y_train_b)

In [75]:
majority_class(X_train_b, y_train_b, X_test_b, y_test_b, le)


=== majority class ===

Classification Report:
                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.00      0.00      0.00        64
                           2. derogation       0.00      0.00      0.00       306
                            3. animosity       0.00      0.00      0.00       231
               4. prejudiced discussions       0.00      0.00      0.00        63
                                    none       0.76      1.00      0.87      2150

                                accuracy                           0.76      2814
                               macro avg       0.15      0.20      0.17      2814
                            weighted avg       0.58      0.76      0.66      2814

Confusion Matrix:
[[   0    0    0    0   64]
 [   0    0    0    0  306]
 [   0    0    0    0  231]
 [   0    0    0    0   63]
 [   0    0    0    0 2150]]


In [76]:
svm_train(X_train_b, y_train_b, X_test_b, y_test_b, le)


=== SVM ===

Classification Report:
                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.18      0.16      0.17        64
                           2. derogation       0.35      0.37      0.36       306
                            3. animosity       0.36      0.27      0.31       231
               4. prejudiced discussions       0.20      0.14      0.17        63
                                    none       0.86      0.89      0.88      2150

                                accuracy                           0.75      2814
                               macro avg       0.39      0.37      0.38      2814
                            weighted avg       0.74      0.75      0.74      2814

Confusion Matrix:
[[  10    7    4    2   41]
 [   8  112   42    7  137]
 [   9   66   62    6   88]
 [   1    9    5    9   39]
 [  27  123   61   22 1917]]


In [77]:
decision_tree_train(X_train_b, y_train_b, X_test_b, y_test_b, le)


=== decision tree ===

Classification Report:
                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.09      0.06      0.07        64
                           2. derogation       0.34      0.29      0.31       306
                            3. animosity       0.39      0.30      0.34       231
               4. prejudiced discussions       0.14      0.11      0.12        63
                                    none       0.85      0.90      0.87      2150

                                accuracy                           0.75      2814
                               macro avg       0.36      0.33      0.35      2814
                            weighted avg       0.72      0.75      0.73      2814

Confusion Matrix:
[[   4    7    6    1   46]
 [   5   89   51    5  156]
 [   7   44   70    7  103]
 [   3    9    6    7   38]
 [  27  111   47   29 1936]]
